In [1]:
import numpy as np
import pandas as pd
import sklearn
import time

In [36]:
import pickle
from matplotlib import pyplot as plt
df = pd.read_pickle('C:/Users/SHIVAM/Downloads/tweet_train_cleaned.pkl')

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
x = df['Tweet']
y = df['Mobile_Tech_Tag']
model = TfidfVectorizer()
model.fit(x)

TfidfVectorizer()

In [38]:
from sklearn.model_selection import train_test_split
data_train_org,data_cv_org,y_train,y_cv = train_test_split(x,y,test_size = 0.1,random_state = 42)

In [39]:
data_train = model.transform(data_train_org)
data_cv = model.transform(data_cv_org)

In [40]:
t0 = time.time()
from sklearn.ensemble import RandomForestClassifier
rfclassifier = RandomForestClassifier(n_jobs = -1)
rfclassifier.fit(data_train,y_train)
tf = time.time()
print("Time taken:" + (str)(tf-t0))

Time taken:0.3550527095794678


In [41]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
y_pred = rfclassifier.predict(data_cv)
print("Weighted F1 Score:")
print(f1_score(y_pred,y_cv,average = 'weighted'))
print("Accuracy Score:")
print(accuracy_score(y_pred,y_cv))

Weighted F1 Score:
0.996781436000636
Accuracy Score:
0.9967741935483871


In [42]:
from sklearn.naive_bayes import GaussianNB
NBclassifier = GaussianNB()
data_train_NB = data_train.toarray()
data_cv_NB = data_cv.toarray()
NBclassifier.fit(data_train_NB, y_train)

GaussianNB()

In [43]:
y_pred = NBclassifier.predict(data_cv_NB)
print("Weighted F1 Score:")
print(f1_score(y_pred,y_cv,average = 'weighted'))
print("Accuracy Score:")
print(accuracy_score(y_pred,y_cv))

Weighted F1 Score:
0.9774700520044511
Accuracy Score:
0.9774193548387097


In [44]:
t0 = time.time()
from xgboost import XGBClassifier
xgbc = XGBClassifier(max_depth = 10,n_estimators = 100,random_state = 42,objective='binary:hinge',n_jobs = -1)
xgbc.fit(data_train,y_train)
tf = time.time()
print("Time taken:" + (str)(tf-t0))

Time taken:0.4657280445098877


In [45]:
y_pred = xgbc.predict(data_cv)
print("Weighted F1 Score:")
print(f1_score(y_pred,y_cv,average = 'weighted'))
print("Accuracy Score:")
print(accuracy_score(y_pred,y_cv))

Weighted F1 Score:
1.0
Accuracy Score:
1.0


In [46]:
t0 = time.time()
from vecstack import StackingTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
estimators = [('Regression',LogisticRegression()),('Naive Bayes',GaussianNB()),
              ('Random Forest',RandomForestClassifier(n_estimators = 50,n_jobs = -1))] 
#('Support Vector',SVC())], #('Decision Tree',DecisionTreeClassifier())]
stack = StackingTransformer(estimators, n_folds = 5, regression = False, random_state = 42, verbose=2)
stack.fit(data_train_NB,y_train)

s_train = stack.transform(data_train_NB)
s_test = stack.transform(data_cv_NB)
model = XGBClassifier(max_depth = 21,max_n_estimators=100,
                         verbosity = 1,random_state = 42,objective = 'binary:hinge',n_jobs = -1)
model.fit(s_train,y_train)
tf = time.time()
print("Time taken:" + (str)(tf-t0))

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
variant:      [A]
n_estimators: [3]

estimator  0: [Regression: LogisticRegression]


C:\Users\SHIVAM\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


    fold  0:  [0.96947935]
    fold  1:  [0.97127469]
    fold  2:  [0.96588869]
    fold  3:  [0.95870736]
    fold  4:  [0.95691203]
    ----
    MEAN:     [0.96445242] + [0.00572258]

estimator  1: [Naive Bayes: GaussianNB]
    fold  0:  [0.98384201]
    fold  1:  [0.98922801]
    fold  2:  [0.97307002]
    fold  3:  [0.96229803]
    fold  4:  [0.96768402]
    ----
    MEAN:     [0.97522442] + [0.00998954]

estimator  2: [Random Forest: RandomForestClassifier]
    fold  0:  [0.98922801]
    fold  1:  [0.99640934]
    fold  2:  [0.99640934]
    fold  3:  [0.99461400]
    fold  4:  [0.99281867]
    ----
    MEAN:     [0.99389587] + [0.00268701]

Train set was detected.
Transforming...

estimator  0: [Regression: LogisticRegression]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    model from fold  4: done
    ----
    DONE

estimator  1: [Naive Bayes: GaussianNB]
    model from fold  0: done
    model from fold  1: 

In [47]:
y_pred = model.predict(s_test)
print(accuracy_score(y_cv,y_pred))
print(f1_score(y_cv,y_pred))

0.9870967741935484
0.974025974025974


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = RandomForestClassifier()
space = dict()
space['n_estimators'] = [100,200,300,400]
space['criterion'] = ['gini', 'entropy']
#space['min_sample_split'] = ['2','4','6']
#space['min_sample_leaf'] = ['1','2','3']
search = RandomizedSearchCV(model, space, n_iter=20, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)
result = search.fit(data_train, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
df = pd.DataFrame(data_cv_org)
df.append(y_pred,'Predicted')
df.append(y_cv,'Actual')
wrong = df['Predicted'] != df['Actual']